In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import numpy as np
import mxnet as mx
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *

No vtk
File does not exist: /media/yuncong/BstemAtlasData/CSHL_data_processed//MD653/MD653_anchor.txt


Setting environment for Precision WorkStation


File does not exist: /media/yuncong/BstemAtlasData/CSHL_data_processed//MD653/MD653_sorted_filenames.txt
File does not exist: /media/yuncong/BstemAtlasData/CSHL_data_processed//MD653/MD653_cropbox.txt
File does not exist: /media/yuncong/BstemAtlasData/CSHL_data_processed//MD653/MD653_cropbox.txt
File does not exist: /media/yuncong/BstemAtlasData/CSHL_data_processed//MD652/MD652_anchor.txt
File does not exist: /media/yuncong/BstemAtlasData/CSHL_data_processed//MD652/MD652_sorted_filenames.txt
File does not exist: /media/yuncong/BstemAtlasData/CSHL_data_processed//MD652/MD652_cropbox.txt
File does not exist: /media/yuncong/BstemAtlasData/CSHL_data_processed//MD652/MD652_cropbox.txt


In [2]:
MODEL_ROOTDIR = '/home/yuncong/mxnet_models/'

In [3]:
# model_dir_name = 'Sat16ClassFinetuned'
# model_name = 'Sat16ClassFinetuned'

# model_dir_name = 'vgg16'
# model_name = 'vgg16'
# model_iteration = 0
# output_symbol_name = 'fc8_output'

# model_dir_name = 'vgg19'
# model_name = 'vgg19'
# model_iteration = 0
# output_symbol_name = 'fc8_output'

# model_dir_name = 'inception-bn'
# model_name = 'Inception-BN'
# model_iteration = 126
# output_symbol_name = 'flatten_output'

model_dir_name = 'inception-bn-sat'
model_name = 'inception-bn-sat'
model_iteration = 0

In [4]:
# mean_img = mx.nd.load(os.path.join(MODEL_ROOTDIR, model_dir_name, 'mean_224.nd'))['mean_img'].asnumpy()
mean_img = np.load(os.path.join(MODEL_ROOTDIR, model_dir_name, 'mean_224.npy'))

In [ ]:
model = mx.model.FeedForward.load(os.path.join(MODEL_ROOTDIR, model_dir_name, model_name), model_iteration, ctx=mx.gpu())

In [ ]:
# model0 = mx.model.FeedForward.load(os.path.join(MODEL_ROOTDIR, model_dir_name, model_name), model_iteration, ctx=mx.gpu())
# flatten_output = model0.symbol.get_internals()[output_symbol_name]
# model = mx.model.FeedForward(ctx=mx.gpu(), 
#                            symbol=flatten_output, 
#                            arg_params = model0.arg_params, 
#                            aux_params = model0.aux_params,
#                            allow_extra_params = True)

In [ ]:
# Visualize network topology

# a = mx.viz.plot_network(flatten_output, shape={"data":(1, 1, 224, 224)}, node_attrs={"shape":'rect',"fixedsize":'false'})
# a.render(model_name)

In [5]:
patch_size = 224
half_size = patch_size/2
stride = 56

In [6]:
# for stack in all_stacks:
for stack in ['MD642']:

    print stack

    section_to_filename = metadata_cache['sections_to_filenames'][stack]

    image_width, image_height = metadata_cache['image_shape'][stack]
    grid_spec = (patch_size, stride, image_width, image_height)

    sample_locations = grid_parameters_to_sample_locations(grid_spec=grid_spec)

    first_detect_sec, last_detect_sec = metadata_cache['section_limits'][stack]

    bar = show_progress_bar(first_detect_sec, last_detect_sec)

    for sec in range(200, 201):
#     for sec in range(first_detect_sec, last_detect_sec+1):

        if is_invalid(stack=stack, sec=sec):
            continue
                
        bar.value = sec

        # Use grids only on mask.
#         t = time.time()
#         mask_tb = DataManager.load_thumbnail_mask_v2(stack=stack, section=sec)
#         indices_roi = locate_patches_v2(grid_spec=grid_spec, mask_tb=mask_tb)
#         sys.stderr.write('locate patches: %.2f seconds\n' % (time.time() - t))       
        
        # Use grids on the entire frame.
        indices_roi = range(len(sample_locations))

        n = len(indices_roi)
        sys.stderr.write('%d samples.\n' % n)

        sample_locations_roi = sample_locations[indices_roi]
        
        t = time.time()
        sat = imread(DataManager.get_image_filepath(stack=stack, section=sec, version='saturation'))
        sys.stderr.write('load saturation image: %.2f seconds\n' % (time.time() - t)) # ~ 35s


        t = time.time()

        patches = np.array([sat[y-half_size:y+half_size, x-half_size:x+half_size]
                            for x, y in sample_locations_roi]) # n x 224 x 224
        patches_mean_subtracted = patches - mean_img
        patches_mean_subtracted_input = patches_mean_subtracted[:, None, :, :] # n x 1 x 224 x 224

        sys.stderr.write('extract patches: %.2f seconds\n' % (time.time() - t)) # ~ 20s / 140k patches

        # Increase batch_size to 500 does not save any time.
        batch_size = 256 
#         batch_size = 16

        # For mxnet 0.9.4, using 80000 patches in a NDArrayIter works but using 90000 patches causes error:
        # TypeError: Invalid type '<type 'numpy.ndarray'>' for data, should be NDArray or numpy.ndarray

        t1 = time.time()
        features = np.empty((n, 1024))
        
        n_each_iter = 80000
        for b in range(0, n, n_each_iter):
            data_iter = mx.io.NDArrayIter(
                patches_mean_subtracted_input[b:b+n_each_iter], 
                np.zeros((n_each_iter, ), np.int), # labels are not important since it is just feed-forward
                batch_size = batch_size,
                shuffle=False)

#             if b == 0:
#                 features = model.predict(data_iter)
#             else:
#                 features = np.vstack([features, model.predict(data_iter)])
            features[b:b+n_each_iter] = model.predict(data_iter)
        sys.stderr.write('predict: %.2f seconds\n' % (time.time() - t1)) # 220s-250s
        
#         data_iter = mx.io.NDArrayIter(
#             patches_mean_subtracted_input, 
#             np.zeros((n, ), np.int), # labels are not important since it is just feed-forward
#             batch_size = batch_size,
#             shuffle=False
#         )

#         t = time.time()
#         features = model.predict(data_iter)
#         sys.stderr.write('predict: %.2f seconds\n' % (time.time() - t))

    # The first time when CUDA needs to load model is very slow ~350s, but later runs are faster ~90s
        
    # Had to modify [model]-symbol.json according to this https://github.com/dmlc/mxnet/issues/2718
    
    # Out of memory after about 20 sections - had to modify storage code according to https://github.com/dmlc/mxnet/issues/3055
    #    - don't think increasing kPoolThreshold to over 4GB will be beneficial, 
    # since the computation time is similar to before limiting the pool, computation is most likely compute-bound.
    # Issue solved by developers.
        
#         t = time.time()

#         features_fp = DataManager.get_dnn_features_filepath(stack=stack, model_name=model_name, 
#                                                           section=sec)
#         create_parent_dir_if_not_exists(features_fp)
#         save_hdf(features.astype(np.float16), features_fp) # bloscpack produces files of similar size

#         feature_locs_fp = DataManager.get_dnn_feature_locations_filepath(stack=stack, model_name=model_name, 
#                                                   section=sec)
#         np.savetxt(feature_locs_fp, np.c_[indices_roi, sample_locations_roi], fmt='%d %d %d')

#         sys.stderr.write('save: %.2f seconds\n' % (time.time() - t)) # ~.5s

139975 samples.
/usr/local/lib/python2.7/dist-packages/skimage/external/tifffile/tifffile.py:1794: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)
load saturation image: 33.71 seconds
extract patches: 14.73 seconds


MD642


NameError: name 'model' is not defined

# Sync with Gordon

In [ ]:
cmd = 'cd /media/yuncong/BstemAtlasData/CSHL_patch_features/Sat16ClassFinetuned/%(stack)s/ && \
rsync -r . yuncong@oasis-dm.sdsc.edu:/home/yuncong/csd395/CSHL_patch_features/Sat16ClassFinetuned/%(stack)s' % \
{'stack': stack}

os.system(cmd)